In [18]:
"""Requirements:
TV Show Search: Scrape or access TV guide websites to search for shows by name, date, and time.
Database: Store the shows, times, and channels in a database for easy access.
Email Reminder: Send an email reminder when the show is about to start.
"""

import requests
from bs4 import BeautifulSoup

def get_tv_show_schedule(show_name):
    url = f'https://www.tvguide.com/search/?searchterm={show_name}'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    
    # Parse relevant info from the page, like show times and channels
    schedule = []
    for show in soup.find_all('div', class_='show-time'):
        time = show.find('span', class_='time').text
        channel = show.find('span', class_='channel').text
        schedule.append({'time': time, 'channel': channel})
    
    return schedule

# Example usage:
show_schedule = get_tv_show_schedule('Game of Thrones')
print(show_schedule)


[]


Create a database connection

In [17]:
import sqlite3


conn = sqlite3.connect('tv_show_tracker.db')
cursor = conn.cursor()

# Create a table to store show schedules
cursor.execute('''
CREATE TABLE IF NOT EXISTS show_schedule (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    show_name TEXT,
    show_time TEXT,
    show_channel TEXT
);
''')

conn.commit()
conn.close()


In [3]:
def save_schedule_to_db(show_name, show_time, show_channel):
    conn = sqlite3.connect('tv_show_tracker.db')
    cursor = conn.cursor()

    cursor.execute('''
    INSERT INTO show_schedule (show_name, show_time, show_channel)
    VALUES (?, ?, ?)
    ''', (show_name, show_time, show_channel))

    conn.commit()
    conn.close()

# Example usage:
for show in show_schedule:
    save_schedule_to_db('Game of Thrones', show['time'], show['channel'])


In [4]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart

def send_email(reminder_to, subject, body):
    sender_email = "your_email@example.com"
    receiver_email = reminder_to
    password = "your_email_password"

    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = receiver_email
    msg['Subject'] = subject

    msg.attach(MIMEText(body, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com', 587)  # Use Gmail's SMTP server
        server.starttls()
        server.login(sender_email, password)
        text = msg.as_string()
        server.sendmail(sender_email, receiver_email, text)
        server.quit()
        print("Email sent successfully!")
    except Exception as e:
        print(f"Failed to send email: {e}")

# Example usage:
send_email('recipient@example.com', 'Reminder: Game of Thrones', 'Your show is about to start on HBO!')


Failed to send email: (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-219dc9f51b5sm173085425ad.184 - gsmtp')


In [5]:
import schedule
import time

def check_and_send_reminders():
    # Fetch shows coming up within the next hour (example)
    conn = sqlite3.connect('tv_show_tracker.db')
    cursor = conn.cursor()

    cursor.execute('''
    SELECT * FROM show_schedule WHERE show_time BETWEEN CURRENT_TIME AND CURRENT_TIME + INTERVAL 1 HOUR;
    ''')

    shows = cursor.fetchall()

    for show in shows:
        show_name = show[1]
        show_time = show[2]
        show_channel = show[3]
        send_email('recipient@example.com', f'Reminder: {show_name}', f'Your show {show_name} is about to start on {show_channel} at {show_time}!')

    conn.close()

# Schedule the check every 30 minutes
schedule.every(30).minutes.do(check_and_send_reminders)

while True:
    schedule.run_pending()
    time.sleep(1)


KeyboardInterrupt: 